In [1]:
import json
import pickle
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [13]:
with open('ftrace\scripts\WITH_ISTIO_WITH_WORKLOAD\1.txt.pid.pickle','rb') as f:
    x = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'ftrace\\scripts\\WITH_ISTIO_WITH_WORKLOAD\x01.txt.pid.pickle'

In [2]:
# GLOBAL VARIABLES / PARAMETERS
WORKLOAD = "WITHOUT_ISTIO_WITHOUT_WORKLOAD"
TRACE_LOGS = f"{WORKLOAD}/"
PARSED_OUTPUTS = f"parsedOutputs/{WORKLOAD}"
NUM_RUNS = 3
FUNC_BLACKLIST = {"BLANK", "do_syscall_64()"}

In [6]:
def get_runs():
    runs = []
    for run_data in os.listdir(PARSED_OUTPUTS):
        with open(os.path.join(PARSED_OUTPUTS, run_data), "r") as fd:
            data = json.load(fd)
            runs.append(data)
    return runs

runs = get_runs()

In [4]:
def get_latency(run, blacklist=FUNC_BLACKLIST):
    latency = {}
    print(blacklist)
    
    def update_latency(data):
        
        for call in data:
            function_name = call['function_name']
            if function_name not in blacklist:              
                latency[function_name] = latency.get(function_name, 0) + call['time']
                
                for child in call['children']:
                    latency[function_name] -= child['time']

            update_latency(call['children'])
    
    for data in run.values():
        update_latency(data)

    return latency

In [8]:
r = runs[0]
pid_to_time = {}
for k,v in r.items():
    pid_to_time[k] = 0
    for call in v:
        pid_to_time[k] += call["time"]
print(pid_to_time)



{'924328': 5089379.733999999, '924356': 1.603, '1616767': 3.737, '54442': 4.137, '3066833': 5019696.016, '3068903': 5516952.423000001, '1561378': 5382093.303000001, '31': 3.968, '3070430': 5819792.061999996, '3069873': 15009604.260000002, '54419': 5.27, '927039': 0.21, '925398': 1.733, '924354': 1.233, '1478': 1.844, '3067873': 11010930.626, '3069253': 10009297.477000002, '1001793': 2.304, '1654183': 3.476, '3068278': 9008751.718, '43': 2.375, '1989488': 3.046, '1664729': 2.274, '1446': 2.905, '1989511': 2.896, '2728': 35021162.23799999}
